In [1]:
from datasets import MeshAirfoilDataset, MeshGeneralDataset
from trainer import Trainer
import os 
class Args:
    pass 
args = Args()
args.case = "airfoil"
args.space_dim=2
args.n_train=2 #1000
args.n_valid= 0 #100
args.n_test=0 #100
args.time_len=1 #500
args.noise_level=[10, 10, 0.01]
# args.shuffle noise level
args.consist_mesh=0
args.multi_mesh_layer=9
args.mp_time=1
args.restart_epoch = -1
args.num_epochs=1 #50
args.batch=1
args.lr=0.0001
args.gamma=0.79432823472
args.noise_gamma = 1
args.hidden_dim = 128
args.hidden_depth = 2
args.data_dir='./data/airfoil'
args.dump_dir='./res/airfoil'
args.recal_mesh = True

/home2/naheed1/miniconda3/envs/cu113py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args.consist_mesh = bool(args.consist_mesh)
args.dump_dir = os.path.join(args.dump_dir, 'ours')
device = 'cuda:0'
trainer = Trainer(args, device)

  0%|                                                                                                | 0/1 [00:00<?, ?iters/s]

In [3]:
trainer.train()

epoch start ------
train  => len =  2
_create_datset_offline:  1 train
field_names:  ['node_type', 'cells', 'mesh_pos', 'density', 'pressure', 'velocity']
field keys:  ['node_type', 'cells', 'mesh_pos', 'density', 'pressure', 'velocity']
self.cells:  (10216, 3)
stride:  1
fields["mesh_pos"].shape[0]:  501
flat_edge.shape:  (2, 30898)
l =  0
n_l =  5233
pos_mesh.shape:  (2638, 2)
g.shape:  (2, 24406)
l =  1
n_l =  2638
pos_mesh.shape:  (1324, 2)
g.shape:  (2, 19922)
l =  2
n_l =  1324
pos_mesh.shape:  (686, 2)
g.shape:  (2, 18214)
l =  3
n_l =  686
pos_mesh.shape:  (357, 2)
g.shape:  (2, 16904)
l =  4
n_l =  357
pos_mesh.shape:  (213, 2)
g.shape:  (2, 21600)
l =  5
n_l =  213
pos_mesh.shape:  (132, 2)
g.shape:  (2, 15972)
l =  6
n_l =  132
pos_mesh.shape:  (67, 2)
g.shape:  (2, 4422)
l =  7
n_l =  67
pos_mesh.shape:  (34, 2)
g.shape:  (2, 1122)
l =  8
n_l =  34
pos_mesh.shape:  (18, 2)
g.shape:  (2, 306)
fields["velocity"] =  torch.Size([500, 5233, 2])
torch.Size([500, 5233, 1])
torch.S

Epoch: 0, Traning RMSE: 15.251622348226752: 100%|████████████████████████████████████████████| 1/1 [00:46<00:00, 46.68s/iters]

mdata =  <class 'datasets.MeshAirfoilDataset'>
epoch end ------


In [26]:
import json,h5py,torch
root = './data/airfoil/'
data_dir = os.path.join(root,'outputs_train/')
instance_id = 9
with open(os.path.join(root, 'meta.json'), 'r') as fp:
    meta = json.loads(fp.read())
field_names = meta['field_names']
print('field_names: ',field_names)
fields = dict()
with h5py.File(os.path.join(data_dir, str(instance_id) + '.h5'), 'r') as f:
    for name in field_names:
        if name == "cells":
            fields[name] = np.array(f[name])
            cells = fields[name][0]
        else:
            fields[name] = torch.tensor(np.array(f[name]), dtype=torch.float)
print(cells.shape)
print(fields["velocity"].shape)

field_names:  ['node_type', 'cells', 'mesh_pos', 'density', 'pressure', 'velocity']
(10216, 3)
torch.Size([501, 5233, 2])


In [4]:
import numpy as np
adj_mat = np.array([[0,1,1],[1,0,0],[1,1,1]])
s, r = np.where(adj_mat.astype(bool))
print(s)
print(r)

[0 0 1 2 2 2]
[1 2 0 0 1 2]


In [5]:
np.where(adj_mat.astype(bool))

(array([0, 0, 1, 2, 2, 2]), array([1, 2, 0, 0, 1, 2]))

In [5]:
import torch
import datasets as cdata
from multiprocessing import Pool
num_threads = 32
def func(id):
    mdata = cdata.ShapeFV('../data/shapes',
                                   instance_id=id,
                                   layer_num=9,
                                   stride=1,
                                   mode = 'train',
                                   recal_mesh=1,
                                   consist_mesh=0,
                                   in_normal_feature_list=['mesh_pos','saf','dsdf'])
# # samples/airfoils
# with Pool(processes=num_threads) as pool:
#     pool.map(func, range(1600))


def func2(id):
    mdata = cdata.ShapeFV('../data/shapes',
                                   instance_id=id,
                                   layer_num=9,
                                   stride=1,
                                   mode = 'valid',
                                   recal_mesh=1,
                                   consist_mesh=0,
                                   in_normal_feature_list=['mesh_pos','saf','dsdf'])
# samples/airfoils
with Pool(processes=num_threads) as pool:
    pool.map(func2, range(200))
    
def func3(id):
    mdata = cdata.ShapeFV('../data/shapes',
                                   instance_id=id,
                                   layer_num=9,
                                   stride=1,
                                   mode = 'test',
                                   recal_mesh=1,
                                   consist_mesh=0,
                                   in_normal_feature_list=['mesh_pos','saf','dsdf'])
# samples/airfoils
with Pool(processes=num_threads) as pool:
    pool.map(func3, range(200))

self.recal_mesh =  1
generate multi-layer
flat_edge.shape:  (2, 13456)
l =  0  index_to_keep.max() =  4785 (2, 10662)
l =  1  index_to_keep.max() =  2394 (2, 7618)
l =  2  index_to_keep.max() =  1198 (2, 6258)
l =  3  index_to_keep.max() =  614 (2, 5628)
l =  4  index_to_keep.max() =  321 (2, 5682)
l =  5  index_to_keep.max() =  172 (2, 7690)
l =  6  index_to_keep.max() =  103 (2, 2756)
l =  7  index_to_keep.max() =  51 (2, 702)
l =  8  index_to_keep.max() =  25 (2, 182)
